In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:

!pip install -q keras
!pip install keras-metrics

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!unzip "/content/gdrive/My Drive/spectrograms_split1.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_3.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_4.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_4.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_5.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00044_5.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_1.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_1.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_2.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_3.wav.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_3.wav_augmented.png  
  inflating: /content/spectrograms_split1/train/hiphop/hiphop.00045_4.wav.png  
  inflating: /conte

In [6]:

import tensorflow.keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tensorflow.keras.metrics


In [7]:
batch_size=64
number_of_class=10
img_rows,img_clns=360,360


In [8]:
!fusermount -u drive
!google-drive-ocamlfuse drive

In [9]:
train_dir='/content/spectrograms_split1/train'
test_dir='/content/spectrograms_split1/test'

In [10]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  label_mode='categorical',
  seed=1337,
  shuffle=True,
  image_size=(360, 360),
  batch_size=batch_size)


test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  label_mode='categorical',
  seed=1337,
  shuffle=True,
  image_size=(360, 360),
  batch_size=batch_size)

Found 9000 files belonging to 10 classes.
Found 1000 files belonging to 10 classes.


In [16]:
model = tensorflow.keras.Sequential()

model.add(tensorflow.keras.layers.Conv2D(128, input_shape=(img_rows,img_clns,3), kernel_size=3, strides=3, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None))

model.add(tensorflow.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None))

model.add(tensorflow.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None))

model.add(tensorflow.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.0001)))
model.add(tensorflow.keras.layers.BatchNormalization())
model.add(tensorflow.keras.layers.Activation('relu'))
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=3, strides=None,name='feature_layer'))




#model.add(tensorflow.keras.layers.Lambda(lambda x : tensorflow.keras.backend.mean(x, axis=1)))
model.add(tensorflow.keras.layers.Flatten())
model.add(tensorflow.keras.layers.Dense(10, activation='softmax'))

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=[['accuracy'],tensorflow.keras.metrics.Recall(),tensorflow.keras.metrics.Precision()])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 120, 120, 128)     3584      
_________________________________________________________________
batch_normalization_8 (Batch (None, 120, 120, 128)     512       
_________________________________________________________________
activation_8 (Activation)    (None, 120, 120, 128)     0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 40, 40, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 40, 40, 128)       147584    
_________________________________________________________________
batch_normalization_9 (Batch (None, 40, 40, 128)       512       
_________________________________________________________________
activation_9 (Activation)    (None, 40, 40, 128)      

In [ ]:
model.fit(train_ds, epochs=20, validation_data=test_ds,shuffle=True) 

Epoch 1/20
141/141 [==============================] - 40s 268ms/step - loss: 1.5749 - accuracy: 0.4613 - recall_2: 0.2809 - precision_2: 0.6526 - val_loss: 8.7821 - val_accuracy: 0.1530 - val_recall_2: 0.1510 - val_precision_2: 0.1589
Epoch 2/20
141/141 [==============================] - 39s 266ms/step - loss: 1.1324 - accuracy: 0.6122 - recall_2: 0.4771 - precision_2: 0.7373 - val_loss: 5.7114 - val_accuracy: 0.1860 - val_recall_2: 0.1790 - val_precision_2: 0.1923
Epoch 3/20
141/141 [==============================] - 39s 265ms/step - loss: 0.9508 - accuracy: 0.6788 - recall_2: 0.5768 - precision_2: 0.7813 - val_loss: 3.0387 - val_accuracy: 0.2720 - val_recall_2: 0.2350 - val_precision_2: 0.2695
Epoch 4/20
141/141 [==============================] - 39s 265ms/step - loss: 0.8104 - accuracy: 0.7339 - recall_2: 0.6510 - precision_2: 0.8157 - val_loss: 1.3645 - val_accuracy: 0.5580 - val_recall_2: 0.4950 - val_precision_2: 0.6157
Epoch 5/20
141/141 [==============================] - 39s 26

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_rows,img_clns),  # all images will be resizes to this size
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical')

test_steps = test_generator.n // batch_size
    

Found 1000 images belonging to 10 classes.


In [15]:
loss, acc,recall,precision= model.evaluate_generator(test_generator, steps=test_steps, verbose=1, workers=3)
print('Testing data   -> loss: %.3f, acc: %.3f' % (loss, acc))
print('Testing data   -> precision: %.3f, recall: %.3f' % (precision, recall))
f1score=(2*precision*recall)/(precision+recall)
print('Testing data   -> f1 score: %.3f' % (f1score))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


15/15 [==============================] - 4s 195ms/step - loss: 4.7471 - accuracy: 0.3229 - recall_1: 0.3135 - precision_1: 0.3212
Testing data   -> loss: 4.747, acc: 0.323
Testing data   -> precision: 0.321, recall: 0.314
Testing data   -> f1 score: 0.317
